In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [5]:
tickers=["AMZN","GOOG","MSFT"]
ohclv_data={}

In [6]:
for ticker in tickers:
    temp=yf.download(ticker,period='7mo',interval='1d')
    temp.dropna(how='any',inplace=True)
    ohclv_data[ticker]=temp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [45]:
def CAGR(DF):
    "function to calculate the Cumulative Annual Growth Rate of a trading strategy"
    df=DF.copy()
    df["return"]=DF["Adj Close"].pct_change()
    df["cum_return"]=(1+df["return"]).cumprod()
    n=len(df)/252
    CAGR=(df["cum_return"][-1])**(1/n)-1
    return CAGR

In [46]:
def volatility(DF):
    "function to calculate annualized volatility of a trading strategy"
    df=DF.copy()
    df["daily_ret"]=DF["Adj Close"].pct_change()
    vol=df["daily_ret"].std()*np.sqrt(252)
    return vol

In [50]:
#Sharpe Ratio
def sharpe(DF,Risk_free_rate):
    df=DF.copy()
    return(CAGR(df)-Risk_free_rate)/volatility(df)

In [51]:
#Sortino Ratio
def sortino(DF,Risk_free_rate):
    df=DF.copy()
    df["return"]=df["Adj Close"].pct_change()
    negative_return=np.where(df["return"]>0,0,df["return"])
    neg_vol=pd.Series(negative_return[negative_return!=0]).std()*np.sqrt(252)
    return(CAGR(df)-Risk_free_rate)/neg_vol

In [52]:
for ticker in ohclv_data:
    print("Sharpe for {}={}".format(ticker,sharpe(ohclv_data[ticker],0.03)))
    print("Sortino for {}={}".format(ticker,sortino(ohclv_data[ticker],0.03)))

Sharpe for AMZN=-0.6420821176967572
Sortino for AMZN=-1.014206274396385
Sharpe for GOOG=-0.454175228146028
Sortino for GOOG=-0.6940832856815521
Sharpe for MSFT=-0.29414496749677266
Sortino for MSFT=-0.4379524804873014
